Sección de imports:


In [ ]:
from google.colab import files
from numpy import true_divide
import pandas as pd
import requests
import json

# Preproceso inicial

Se ha decidido trabajar con los municipios de Madrid ya que se encontró una API (vía michelín) para estos que resultaba útil. Así que en primer lugar se filtran los registros de blablacar cuyo destino y origen es un municipio de Madrid con una lista de estos:


Esta lista se ha obtenido scrapeando una tabla de wikipedia de los municipios de Madrid:*

*Hay que destacar que el código mostrado en los primeros bloques NO está preparado para ejecutarse con colab, ya que fue algo hecho con python nativo, y por lo tanto no es recomendable su ejecución desde colab.

In [ ]:
table = pd.read_html('https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_Comunidad_de_Madrid')

ciudades = table[0]['Nombre']

with open('ciudades.txt', 'a+', encoding="utf-8") as f:
    for i in ciudades:
        f.write(i+'\n')

Ya que algunos municipios tenían un formato diferente (el artículo del nombre del municipio se encontraba desposicionado y con paréntesis), lo cambiamos a mano para que los nombres coincidieran en el filtrado.

Después de esto se realiza el filtrado:

In [ ]:
ciudades = []
with open("ciudades.txt", 'r',) as tf:
    lines = tf.read().split('\n')
    
for line in lines:
    ciudades.append(line)

df_bla = pd.read_csv('viajes.txt',skiprows=[1], sep="|", encoding='utf-8', header=None)
df_bla = df_bla.drop(columns = [0, 1, 4, 9, 10], axis = 1)
df_bla = df_bla.drop([0], axis= 0)
df_bla = df_bla.rename(columns={2: "origen", 3: "destino", 6:"asientos", 5:"a_ofertados", 7:"v_ofertados", 8:"viajes"})
df_bla = df_bla.dropna()
df_bla = df_bla[(df_bla.origen.isin(ciudades)) & df_bla.destino.isin(ciudades)]
df_bla = df_bla.reset_index(drop=True)
df_bla.to_csv('rutas_madrid.csv', encoding='utf-8')

Como era necesario saber las coordenadas de cada municipio, se intentaron sacar con el módulo geopy, pero debido a que era necesario relizar muchas solicitudes, este se bloqueaba y no permitía realizar más, así que estas fueron scrapeadas también, y con ellas se creó un diccionario para revisar siempre que fuera necesario saberlas:*

*Este programa también fue ejecutado en python nativo, así que tampoco es recomendable su ejecución en colab.

In [ ]:
table = pd.read_html('https://www.coordenadas.com.es/espana/pueblos-de-madrid/28/1')

with open("coord.txt", 'w') as output:
    for row in table:        
        output.write(str(row) + '\n')

Con la cordenadas scrapeadas se procede a crear un diccionario para consultar en cualquier momento que sea necesario:

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving rutas_madrid.csv to rutas_madrid (1).csv
Saving coord.csv to coord.csv
Saving concentracion_accidentes.csv to concentracion_accidentes.csv
User uploaded file "rutas_madrid.csv" with length 4291726 bytes
User uploaded file "coord.csv" with length 6377 bytes
User uploaded file "concentracion_accidentes.csv" with length 251 bytes


In [ ]:
df_coord = pd.read_csv('coord.csv', sep=';')
df_rutas = pd.read_csv('rutas_madrid.csv')

df_tarjeta = df_rutas.groupby(['origen','destino'], as_index=False).aggregate({'a_ofertados': 'mean','v_ofertados':'mean'})

cd = {}
for element in df_coord.values:
    cd[element[0]] = [float(element[1].replace(',','.')), float(element[2].replace(',','.'))]

Debido a un error obtenido en el fragmento de código de las peticiones (había un municipio cuyas coordenadas no estaban disponibles, ya fuera por un typo en el nombre del municipio o por falta de datos en el csv de las coordenadas), se realizó este trozo de código para eliminar algunos registros y evitar errores.


In [ ]:
municipios_no_coord = []
for ruta in df_tarjeta.values:
  try:
    c_origen = cd[ruta[0]]
  except KeyError:
    municipios_no_coord.append(ruta[0])
  try:
    c_destino = cd[ruta[1]]
  except:
    municipios_no_coord.append(ruta[1])

for element in municipios_no_coord:
  df_tarjeta = df_tarjeta[df_tarjeta["origen"].str.contains(element)==False]
  df_tarjeta = df_tarjeta[df_tarjeta["destino"].str.contains(element)==False]
  print("Líneas que contienen:",element,"han sido eliminadas.")


Líneas que contienen: San Agustín del Guada han sido eliminadas.


Sabiendo las coordenadas de cada municipio y teniendo los trayectos, ya se pueden realizar solicitudes a la API que ha sido mencionada al principio para obtener las carreteras, distancias y tiempos: *

*La ejecución de las requests es MUY lenta ya que son muchas, si es necesario ejecutar este primer bloque, asegurarse de que solo se realice una vez, tarda alrededor de unos 40min.

In [ ]:
peticiones = []
municipios_a_eliminar = []
i = 0
for ruta in df_tarjeta.values:
  c_origen = cd[ruta[0]]
  c_destino = cd[ruta[1]]
  url = (
        'https://secure-apir.viamichelin.com/apir/1/route.json/spa?steps=1:e:{}:{};1:e:{}:{}&multipleIt=true&callback=true&authkey=RESTGP20211210013403787883414137'
    ).format(c_origen[1], c_origen[0], c_destino[1], c_destino[0])
  peticiones.append(requests.get(url).content.decode('iso-8859-1'))

In [ ]:
distancias = [] 
tiempos = []
carreteras = []

for peticion in peticiones:
  
  ruta = json.loads(peticion[5:(len(peticion)-1)])
  path = ruta['iti']['header']['summaries']
  for ruta in path:
    distancias.append(ruta['drivingDist'])
    tiempos.append(ruta['drivingTime'])
    carreteras.append(','.join(ruta['names']))
    
df_tarjeta['distancia'] = distancias
df_tarjeta['tiempo'] = tiempos
df_tarjeta['carreteras'] = carreteras

ruta = json.loads(peticiones[0][5:(len(peticiones[0])-1)])
print(ruta)
#df_tarjeta.to_csv('pre_tarjeta', sep='\t', encoding='utf-8')

{'iti': {'header': {'vehicle': 0, 'itiType': 0, 'idx': 0, 'itidate': '2021-12-19T16:54:03.938Z', 'startMapDef': {'id': '-t44aay3br3h1md', 'size': {'w': 300, 'h': 300}}, 'destMapDef': {'id': '-t4wjgdpbn1h1md', 'size': {'w': 300, 'h': 300}}, 'summaries': [{'index': 0, 'names': ['M-100', 'Avenida de Daganzo'], 'totalDist': 14687.0, 'totalTime': 1122.0, 'motorwayDist': 6740.0, 'motorwayTime': 310.0, 'pleasantDist': 0.0, 'pleasantTime': 0.0, 'drivingDist': 14687.0, 'drivingTime': 1122.0, 'consumption': 1.99, 'carbonFootprintEstimation': 3.52, 'tollCost': {'moto': 0.0, 'car': 0.0, 'caravan': 0.0, 'pl2': 0.0, 'pl3': 0.0, 'pl4': 0.0, 'pl5': 0.0}, 'CCZCost': {'moto': 0.0, 'car': 0.0, 'caravan': 0.0, 'pl2': 0.0, 'pl3': 0.0, 'pl4': 0.0, 'pl5': 0.0}, 'fullMapDef': {'id': '-tutusvn6x0h1md', 'size': {'w': 600, 'h': 600}}, 'ecoTax': 0.0, 'ecoTaxDist': 0.0, 'ecoTaxRepercussionRate': 0.0, 'distanceByCountry': [], 'avoidClosedRoadUsed': False, 'eventTrafficDatabaseAvailable': False}]}, 'itineraryTrace':

Ahora se procede a añadir el número de tramos (diferentes carreteras) que es necesario tomar para llegar a un destino: 

In [ ]:
lista_tramos = []

for seccion in df_tarjeta['carreteras'].values:
  lista_tramos.append(len(seccion.split(',')))

df_tarjeta['n_tramos'] = lista_tramos

df_tarjeta.head()

,origen,destino,a_ofertados,v_ofertados,distancia,tiempo,carreteras,n_tramos
0,Ajalvir,Alcalá de Henares,2.5,1.0,14687.0,1122.0,"M-100,Avenida de Daganzo",2
1,Ajalvir,Algete,3.0,1.0,9353.0,709.0,"M-114,M-118,M-103",3
2,Ajalvir,Arganda del Rey,3.0,1.0,36872.0,1943.0,"M-50,R-3",2
3,Ajalvir,Brunete,3.0,1.0,58267.0,3414.0,"R-2,M-40,M-513",3
4,Ajalvir,Cobeña,1.5,1.0,5197.0,420.0,M-114,1


Finalmente, se indica que viajes tienen una zona de concentración de accidentes. Para esto se ha obtenido un pequeño CSV copiado a mano (ya que solo son 15 registros) de http://mapamovilidad.dgt.es/home.htm y se compara con las carreteras de cada viaje:

In [ ]:
df_accidentes = pd.read_csv('concentracion_accidentes.csv',sep=';')

df_accidentes = df_accidentes.drop_duplicates(subset=['carretera'])

accidentes_dic = {}
lista_accidentes = []

for elemento in df_accidentes['carretera'].values:
  accidentes_dic[elemento] = ""

for seccion in df_tarjeta['carreteras'].values:
  accidente = False
  for carretera in seccion.split(','):
    if carretera in accidentes_dic:
      accidente = True
  if accidente:
    lista_accidentes.append("si")
  else:
    lista_accidentes.append("no")


df_tarjeta['riesgo'] = lista_accidentes


Se añade la velocidad media:

In [ ]:
df_tarjeta['velocidad_media'] = (df_tarjeta['distancia']/1000)/(df_tarjeta['tiempo']/3600)
df_tarjeta.head()

,origen,destino,a_ofertados,v_ofertados,distancia,tiempo,carreteras,n_tramos,riesgo,velocidad_media
0,Ajalvir,Alcalá de Henares,2.5,1.0,14687.0,1122.0,"M-100,Avenida de Daganzo",2,no,47.12
1,Ajalvir,Algete,3.0,1.0,9353.0,709.0,"M-114,M-118,M-103",3,no,47.49
2,Ajalvir,Arganda del Rey,3.0,1.0,36872.0,1943.0,"M-50,R-3",2,si,68.32
3,Ajalvir,Brunete,3.0,1.0,58267.0,3414.0,"R-2,M-40,M-513",3,si,61.44
4,Ajalvir,Cobeña,1.5,1.0,5197.0,420.0,M-114,1,no,44.55


Por último, se realizan aproximaciones, se eliminan filas no válidas y se imprime la tarjeta de datos final:

In [ ]:
df_tarjeta = df_tarjeta.round({"a_ofertados":2,"v_ofertados":2,"velocidad_media":2})
df_tarjeta = df_tarjeta[df_tarjeta['origen'] != df_tarjeta['destino']]
df_tarjeta.to_csv('tarjeta.csv', sep=';', encoding='utf-8',index=False)